## This notebook shows how to use Orchestrator APIs for user experiments

In [ ]:
import os
from fabrictestbed.slice_manager import SliceManager, Status, SliceState
import json

In [ ]:
slice_name='Slice-fp'

In [ ]:
fabric_rc_location=os.environ['HOME']+"/work/fabric_config/fabric_rc"
if os.path.exists(fabric_rc_location):
    with open(fabric_rc_location, 'r') as f:
        for line in f:
            if line.startswith('export'):
                os.environ[line.split('=')[0].split('export')[1].strip()] = line.split('=')[1].strip()

In [ ]:
ssh_key_file_priv=os.environ["FABRIC_SLICE_PRIVATE_KEY_FILE"]
ssh_key_file_pub=os.environ["FABRIC_SLICE_PUBLIC_KEY_FILE"]

ssh_key_pub = None
with open (ssh_key_file_pub, "r") as myfile:
    ssh_key_pub=myfile.read()
    ssh_key_pub=ssh_key_pub.strip()

## Create Slice Manager Object
Users can request tokens with different Project and Scopes by altering `project_name` and `scope` parameters in the refresh call below.

In [ ]:
slice_manager = SliceManager()

### Orchestrator API example to query for available resources

In [ ]:
status, advertised_topology = slice_manager.resources()

print(f"Status: {status}")
if status == Status.OK:
    print(f"Toplogy: {advertised_topology}")
else:
    print(f"Error: {advertised_topology}")

In [ ]:
if status == Status.OK:
    advertised_topology.draw()

## Create Slice

In [ ]:
from fabrictestbed.slice_editor import ExperimentTopology, Capacities, Labels, ComponentType, ComponentModelType, ServiceType
# Create topology
t = ExperimentTopology()



# FP <--> FP Connectivity
'''
fac2 = t.add_facility(name='RENC-GSU', site='RENC', capacities=Capacities(bw=10),
                      labels=Labels(vlan='1000'))

# Uncomment for Facility Port <-> Facility Port connections
fac1 = t.add_facility(name='RENCI-Chameleon', site='RENC', capacities=Capacities(bw=10),
                      labels=Labels(vlan='2000'))

# facility needs to be connected via a service to something else
sfac = t.add_network_service(name='s-fac', nstype=ServiceType.L2Bridge, interfaces=[fac1.interface_list[0], fac2.interface_list[0]])

'''
# VM -> FP Connections

n = t.add_node(name="n1", site="UKY")
cap = Capacities(core=2, ram=6, disk=10)
n.set_properties(capacities=cap, image_type='qcow2', image_ref='default_rocky_8')
n.add_component(model_type=ComponentModelType.SharedNIC_ConnectX_6, name="nic1")

fac2 = t.add_facility(name='RENC-GSU', site='RENC', capacities=Capacities(bw=10),
                      labels=Labels(vlan='1000'))

sfac = t.add_network_service(name='s-fac', nstype=ServiceType.L2STS, interfaces=[n.interface_list[0],
                                                                                 fac2.interface_list[0]])

    
# Generate Slice Graph
slice_graph = t.serialize()

# Request slice from Orchestrator
status, reservations = slice_manager.create(slice_name=slice_name, slice_graph=slice_graph, ssh_key=ssh_key_pub)

print("Response Status {}".format(status))
if status == Status.OK:
    print("Reservations created {}".format(reservations))
    slice_id = reservations[0].slice_id
else:
    print(f"Failure: {reservations}")

## Query Slices

In [ ]:
status, slices = slice_manager.slices()

print("Response Status {}".format(status))
if status == Status.OK:    
    slice_object=list(filter(lambda s: s.slice_id == slice_id, slices))[0]
    print(slice_object)
else:
    print(f"Failure: {slices}")

## Query Slivers

In [ ]:
status, slivers = slice_manager.slivers(slice_object=slice_object)

print("Response Status {}".format(status))
if status == Status.OK:
    for s in slivers:
        print("=====================")
        print("ReservationId {}".format(s.reservation_id))
        print("State         {}".format(s.reservation_state))
        print("Pending       {}".format(s.pending_state))
        print("Join State    {}".format(s.join_state))
        print("Sliver Type   {}".format(s.sliver_type))
        print("Sliver        {}".format(s.sliver))
        if s.sliver_type == "NodeSliver":
            for d in s.sliver.attached_components_info.devices.values():
                print("-------------------------")
                print("COMPONENT        {}".format(d))
                print("-------------------------")
        else:
            for d in s.sliver.interface_info.interfaces.values():
                print("-------------------------")
                print("INTERFACE        {}".format(d))
                print("-------------------------")
                
        print("Notices       {}".format(s.notices))
        print("=====================")
else:
    print(f"Failure: {slivers}")

## Delete Slice

In [ ]:
status, result = slice_manager.delete(slice_object=slice_object)

print("Response Status {}".format(status))
print("Response received {}".format(result))